In [4]:
import pandas as pd
from collections import defaultdict
df = pd.read_csv(r"Real_Data\primaryschool.csv", sep="\t", header=None)

merged = defaultdict(set)

for node_col, attr_col in [(1, 3), (2, 4)]:
    for node, attrs in df.groupby(node_col)[attr_col]:
        merged[node].update(attrs)

meta_df = (
    pd.DataFrame(
        [(node, next(iter(attrs))) for node, attrs in merged.items()],
        columns=["node", "attribute"],
    )
    .sort_values("attribute")
    .reset_index(drop=True)
)

node_mapping = dict(zip(meta_df["node"], meta_df.index))
node_to_attribute = dict(zip(meta_df.index, meta_df["attribute"]))

df = df[[0, 1, 2]]
df.columns = ["starting_times", "source_nodes", "target_nodes"]

df["source_nodes"] = df["source_nodes"].map(node_mapping)
df["target_nodes"] = df["target_nodes"].map(node_mapping)


MIN_TIME = 8.5 * 3600
df["starting_times"] = df["starting_times"] - MIN_TIME
df = df[df["starting_times"] > 24 * 3600].reset_index(drop=True)
df["starting_times"] = df["starting_times"] - 24 * 3600
df['starting_times']=df['starting_times']-df['starting_times'].min()
df["ending_times"] = df["starting_times"] + 20
df['durations']=df["ending_times"] -df["starting_times"] 
df.to_csv('Pre_processed_Data\primary_school_day2.csv', index=False)